In [1]:
import pandas as pd 
import numpy as np
import math
import keras
import tensorflow as tf
import progressbar
import os
from os import listdir

## Print Dependencies



Dependences are fundamental to record the computational environment.

In [2]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p pandas,keras,numpy,math,tensorflow,matplotlib,h5py

# date
print (" ")
%watermark -u -n -t -z

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.22.0

pandas    : 1.2.3
keras     : 2.4.3
numpy     : 1.19.5
math      : unknown
tensorflow: 2.4.1
matplotlib: 3.4.0
h5py      : 2.10.0

Compiler    : Clang 12.0.0 (clang-1200.0.32.29)
OS          : Darwin
Release     : 19.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

 
Last updated: Fri Apr 23 2021 13:40:32CEST



## Load of the data

   You can also load all of them! Writing "all_data"

In [3]:
from process import loaddata
class_data = loaddata("../data_old/{}.csv".format('probability'))

In [4]:
class_data = class_data[class_data[:,0] > 0.00001]

In [23]:
np.random.shuffle(class_data)
y = class_data[:,0]
A = class_data
print(A[0])
A[:,9] = A[:,13]
print(A[0])
x = class_data[:,1:10]
print(x[0])
print(x.shape)

[ 3.78494936e-02 -4.93262316e-01  4.89205911e-01  3.30730208e-01
 -6.17692860e-02 -1.28659625e-01  5.39618230e-01  8.20000000e-01
  5.40000000e+08  4.32593857e-03  4.64616793e-01 -4.60094173e-01
  1.51497925e-02  4.32593857e-03]
[ 3.78494936e-02 -4.93262316e-01  4.89205911e-01  3.30730208e-01
 -6.17692860e-02 -1.28659625e-01  5.39618230e-01  8.20000000e-01
  5.40000000e+08  4.32593857e-03  4.64616793e-01 -4.60094173e-01
  1.51497925e-02  4.32593857e-03]
[-4.93262316e-01  4.89205911e-01  3.30730208e-01 -6.17692860e-02
 -1.28659625e-01  5.39618230e-01  8.20000000e-01  5.40000000e+08
  4.32593857e-03]
(135922, 9)


In [31]:
x.shape

(135922, 9)

In [32]:
y.shape

(135922,)

## Model Build

In [24]:
from keras.models import Sequential
from keras.layers.core import Dense
import keras.backend as K
from keras import optimizers
from keras import models
from keras import layers
from keras.layers.normalization import BatchNormalization

In [25]:
train_split = 0.75
train_limit = int(len(y)*train_split)
print("Training sample: {0} \nValuation sample: {1}".format(train_limit, len(y)-train_limit))

Training sample: 101941 
Valuation sample: 33981


In [26]:
x_train = x[:train_limit]
x_val = x[train_limit:]

y_train = y[:train_limit]
y_val = y[train_limit:]

In [27]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split 
from keras import Sequential 
from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D

In [28]:
from keras.models import Sequential
from keras.layers.core import Dense
import keras.backend as K
from keras import optimizers
from keras import models
from keras import layers
from keras.layers.normalization import BatchNormalization

In [44]:
def build_model():
    model_lstm = Sequential()
    model_lstm.add(Embedding(9,1))
    model_lstm.add(SpatialDropout1D(0.3))
    model_lstm.add(LSTM(256, dropout = 0.3, recurrent_dropout = 0.3))
    model_lstm.add(Dense(256, activation = 'sigmoid'))
    model_lstm.add(Dropout(0.3))
    model_lstm.add(Dense(256, activation = 'sigmoid'))
    model_lstm.add(Dense(9, activation = 'relu'))
    model_lstm.compile(
        loss='mse',
        optimizer='Adam',
        metrics=['accuracy']
    )
    return model_lstm

In [45]:
model_lstm = build_model()
history = model_lstm.fit ( 
    x_train, y_train, epochs = 10, batch_size = 100000
)
model_stm.save("../models/classifier/{}.h5".format('lstm'))

Epoch 1/10


InvalidArgumentError:  indices[25000,7] = 2140000000 is not in [0, 9)
	 [[node sequential_8/embedding_4/embedding_lookup (defined at <ipython-input-45-d67e1be5efcf>:2) ]] [Op:__inference_train_function_13267]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_8/embedding_4/embedding_lookup:
 sequential_8/embedding_4/embedding_lookup/11501 (defined at /Users/chiara/.pyenv/versions/3.8.8/lib/python3.8/contextlib.py:113)

Function call stack:
train_function


In [ ]:
model_lstm.summary()

In [ ]:
import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']

accuracy = history.history['mape']
val_accuracy = history.history['val_mape']


epochs = range(1, len(loss) + 1)
fig, ax1 = plt.subplots()

l1 = ax1.plot(epochs, loss, 'bo', label='Training loss')
vl1 = ax1.plot(epochs, val_loss, 'b', label='Validation loss')
ax1.set_title('Training and validation loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss (mae))')

ax2 = ax1.twinx()
ac2= ax2.plot(epochs, accuracy, 'o', c="red", label='Training acc')
vac2= ax2.plot(epochs, val_accuracy, 'r', label='Validation acc')
ax2.set_ylabel('mape')

lns = l1 + vl1 + ac2 + vac2
labs = [l.get_label() for l in lns]
ax2.legend(lns, labs, loc="center right")
fig.tight_layout()
#fig.savefig("acc+loss_drop.pdf")
fig.show()

## Probability density distribution

In [ ]:
y

In [ ]:
y_pred = model.predict(x)

In [ ]:
y_pred

In [ ]:
def probability(prob_array, bins):
    prob_array = np.array(prob_array)
    plt.hist(prob_array, bins, histtype=u'step', density=True)
    plt.show()

In [ ]:
from matplotlib import pyplot
y = np.array(y)
bins = np.linspace(0.0001, 0.006, 100)
pyplot.hist(y, bins, color = 'indianred', alpha=0.5, label='Osiris')
#pyplot.hist(y_pred, bins, color = 'mediumslateblue', alpha=0.5, label='NN')
pyplot.yscale('log')
pyplot.legend(loc='upper right')
pyplot.xlabel('Probability')
pyplot.title('Trained on ($p_e$, $p_{\gamma}$, $\omega_e$, $\omega_{\gamma}$, n)')
pyplot.show()